# BASE CODE

In [240]:
# Importing the libraries
import re
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Extra libraries
import json
import googlemaps
from datetime import datetime

with open('creds.json') as f:
    api_key = json.load(f)

gmaps = googlemaps.Client(key=api_key['GOOGLE_MAPS_API_KEYS'])

# API KEY    
## api_key['GOOGLE_MAPS_API_KEYS']

In [241]:
df = pd.read_csv("data/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv") # OR USE searchData/limitData
df = df.drop_duplicates()
df = df.sample(frac=0.0005, random_state=1) # GETS SMALL SMAPLE OF ROWS
df = df.reset_index()
print(df.dtypes)
print("Length:", len(df))
df.head(2)

index                    int64
month                   object
town                    object
flat_type               object
block                   object
street_name             object
storey_range            object
floor_area_sqm         float64
flat_model              object
lease_commence_date      int64
remaining_lease         object
resale_price           float64
dtype: object
Length: 74


,index,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,102277,2021-07,HOUGANG,5 ROOM,408,HOUGANG AVE 10,10 TO 12,121.0,Improved,1986,63 years 10 months,580000.0
1,44647,2019-02,TAMPINES,4 ROOM,864A,TAMPINES ST 83,13 TO 15,104.0,Model A,1988,68 years 06 months,450000.0


In [242]:
df['year'] = df['month'].apply(lambda x: x.split('-')[0]).astype(int)
df['month'] = df['month'].apply(lambda x: x.split('-')[1]).astype(int)

# Convert remaining lease from years to months
df['month_remaining_lease'] = df['remaining_lease'].apply(lambda x: int(x.split(' ')[0]) * 12 + int(x.split(' ')[2]) if len(x.split(' ')) > 2 else int(x.split(' ')[0]) * 12)

# Split storey range into min and max
df['storey_range_low'] = df['storey_range'].apply(lambda x: int(x.split('TO')[0]))
df['storey_range_high'] = df['storey_range'].apply(lambda x: int(x.split('TO')[1]))

# Convert categorical columns to numerical
df['flat_model'] = df['flat_model'].astype('category').cat.codes

df[['month', 'year', 'town', 'flat_type', 'floor_area_sqm', 'lease_commence_date', 'month_remaining_lease', 'flat_model', 'storey_range_low', 'storey_range_high', 'resale_price']].head(5)

,month,year,town,flat_type,floor_area_sqm,lease_commence_date,month_remaining_lease,flat_model,storey_range_low,storey_range_high,resale_price
0,7,2021,HOUGANG,5 ROOM,121.0,1986,766,3,10,12,580000.0
1,2,2019,TAMPINES,4 ROOM,104.0,1988,822,5,13,15,450000.0
2,7,2021,BUKIT BATOK,3 ROOM,73.0,1985,751,5,7,9,357000.0
3,5,2018,PUNGGOL,4 ROOM,92.0,2014,1141,5,13,15,463000.0
4,11,2021,WOODLANDS,4 ROOM,106.0,1995,877,5,4,6,420000.0


# ADD ON COLUMNS CODE

## 1. Get walking distance to nearest MRT station

In [282]:
def get_nearest_mrt_location(addr, location_type):
    # Request directions via public transit
    # Returns Start, End, Total distance

    coords = gmaps.geocode(f"{addr}, Singapore")
    
    lat = coords[0]['geometry']['location']['lat']
    lng = coords[0]['geometry']['location']['lng']

    stn_found = False
    radius = 100

    while not stn_found:
        nearest_location = gmaps.places_nearby(location=f"{lat}, {lng}", keyword=location_type, rank_by="distance")
        if len(nearest_location['results']) > 0:
            stn_found = True
        else:
            radius += 100
    
    if "MRT" in nearest_location['results'][0]['name']:
        if "Singapore" in nearest_location['results'][0]['name']:
            place_addr = place_addr = nearest_location['results'][0]['name']
        else:
            place_addr = place_addr = nearest_location['results'][0]['name'] + ", Singapore"
    else:
        place_addr = place_addr = nearest_location['results'][0]['name'] + " MRT Station, Singapore"
        
    if len(place_addr) == 0:
        raise Exception("No MRT station found")
    
    print(nearest_location)
    print("START:", f"{addr}")
    print("END:", place_addr)
    
    directions_result = gmaps.distance_matrix(addr, place_addr, mode="walking")
    print("DISTANCE:", directions_result['rows'][0]['elements'][0]['distance']['text'])
    print("\n")

    return  directions_result['rows'][0]['elements'][0]['distance']['text'].split(' ')[0]

In [283]:
get_nearest_mrt_location("627 PASIR RIS DR 3, Singapore", "mrt")

{'html_attributions': [], 'next_page_token': 'AfLeUgMXJXbVbKTSpbmPFOfVbtRuVsi77ynXrNEaTOwx2u144uECf9XMJdxnxxEtP80vQqsODoWGnBtPULU7n6K8pl9L6TwSUAgMSPY8ZWuZne3JlQ6bmdBm6SOdmzsIwA8BNpL5oXPhd_vSnNBxFmCScYhT6NBhg5XQgPV1-ld_fviCll2cYh0Fkc5mcEIRmwjHJ-Wb-PRKnsrtXriGPjD-FPRA1CXIoeRtK0fUCBTs4jiH4AdzKnqtlzNNOWaGhMtb9mtzzXjc-E9MI9hrleUu5m-WPLi852lnDKmRWAvQaV8BAtcuiPVMv2yUNLD3OrCutzILX9D0zhWgpAnr6vjb7Gg9uoj3kHnck4DtXQfH1DyyHjekA07kiPinBTb2UlL5bsXVZg-GhPjJ4AA4FuSHaVaCRcIYcXNcSJaQATT_rvmKoQGvLFxXug', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 1.369232, 'lng': 103.9385347}, 'viewport': {'northeast': {'lat': 1.369917229892722, 'lng': 103.9399479798927}, 'southwest': {'lat': 1.367217570107278, 'lng': 103.9372483201073}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png', 'icon_background_color': '#7B9EB0', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet', 'name': 'Tampines North M

'3.4'

In [280]:
df['dist_to_nearest_stn_walking'] = df.apply(lambda row : get_nearest_mrt_location(f"{row['block']} {row['street_name']}, Singapore", "mrt"), axis = 1)

START: 408 HOUGANG AVE 10, Singapore
END: Hougang MRT Station, Singapore
DISTANCE: 0.6 km


START: 864A TAMPINES ST 83, Singapore
END: Tampines MRT Station (DT32), Singapore
DISTANCE: 1.1 km


START: 127 BT BATOK WEST AVE 6, Singapore
END: BUkit Batok MRT Station, Singapore
DISTANCE: 0.7 km


START: 208B PUNGGOL PL, Singapore
END: Punggol MRT Station, Singapore
DISTANCE: 0.5 km


START: 809 WOODLANDS ST 81, Singapore
END: Woodlands North MRT Station, Singapore
DISTANCE: 0.9 km


START: 108 YISHUN RING RD, Singapore
END: Yishun MRT Station, Singapore
DISTANCE: 1.0 km


START: 770 WOODLANDS DR 60, Singapore
END: Admiralty MRT Station, Singapore
DISTANCE: 0.8 km


START: 420 JURONG WEST ST 42, Singapore
END: Lakeside MRT Station, Singapore
DISTANCE: 1.4 km


START: 234 JURONG EAST ST 21, Singapore
END: Toh Guan MRT Station (JE4) (U/C), Singapore
DISTANCE: 0.3 km


START: 462 CRAWFORD LANE, Singapore
END: Lavender MRT Station, Singapore
DISTANCE: 0.4 km


START: 8 JLN BT HO SWEE, Singapore

## 2. Get walking distance to nearest supermarket

In [ ]:
def get_nearest_sp_location(addr, location_type):
    # Request directions via public transit
    # Returns Start, End, Total distance

    coords = gmaps.geocode(f"{addr}, Singapore")
    
    lat = coords[0]['geometry']['location']['lat']
    lng = coords[0]['geometry']['location']['lng']

    stn_found = False
    radius = 100

    while not stn_found:
        nearest_location = gmaps.places(location=f"{lat}, {lng}", query=location_type, radius=radius)
        if len(nearest_location['results']) > 0:
            stn_found = True
        else:
            radius += 100
    
    directions_result = gmaps.directions(f"{addr}",
                                     nearest_location['results'][0]['formatted_address'],
                                     mode="walking",
                                     departure_time=datetime.now())[0]
    
    '''
    print("START:", f"{addr}")
    print("END:", nearest_location['results'][0]['name'], ",", nearest_location['results'][0]['formatted_address'])
    print("DISTANCE:", directions_result['legs'][0]['distance']['text'])
    print("\n")
    '''
    
    return directions_result['legs'][0]['distance']['text'].split(' ')[0]

In [284]:
df['dist_to_nearest_supermarket_walking'] = df.apply(lambda row : get_nearest_sp_location(f"{row['block']} {row['street_name']}, Singapore", "supermarket"), axis = 1)

In [ ]:
df.head(20)

,index,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,year,month_remaining_lease,storey_range_low,storey_range_high,dist_to_nearest_stn_walking,dist_to_nearest_supermarket_walking
0,102277,7,HOUGANG,5 ROOM,408,HOUGANG AVE 10,10 TO 12,121.0,3,1986,63 years 10 months,580000.0,2021,766,10,12,0.6,0.3
1,44647,2,TAMPINES,4 ROOM,864A,TAMPINES ST 83,13 TO 15,104.0,5,1988,68 years 06 months,450000.0,2019,822,13,15,1.1,0.2
2,101607,7,BUKIT BATOK,3 ROOM,127,BT BATOK WEST AVE 6,07 TO 09,73.0,5,1985,62 years 07 months,357000.0,2021,751,7,9,0.7,0.7
3,27448,5,PUNGGOL,4 ROOM,208B,PUNGGOL PL,13 TO 15,92.0,5,2014,95 years 01 month,463000.0,2018,1141,13,15,1.9,0.3
4,113957,11,WOODLANDS,4 ROOM,809,WOODLANDS ST 81,04 TO 06,106.0,5,1995,73 years 01 month,420000.0,2021,877,4,6,0.9,0.3
5,34735,8,YISHUN,4 ROOM,108,YISHUN RING RD,04 TO 06,91.0,6,1985,65 years 05 months,330000.0,2018,785,4,6,1.0,0.3
6,147484,1,WOODLANDS,4 ROOM,770,WOODLANDS DR 60,04 TO 06,103.0,5,1997,73 years 11 months,380000.0,2023,887,4,6,0.8,0.3
7,31262,7,JURONG WEST,4 ROOM,420,JURONG WEST ST 42,07 TO 09,84.0,8,1985,65 years 10 months,305000.0,2018,790,7,9,1.4,0.3
8,86116,12,JURONG EAST,5 ROOM,234,JURONG EAST ST 21,04 TO 06,121.0,3,1982,60 years 02 months,478000.0,2020,722,4,6,1.2,0.5
9,10586,7,KALLANG/WHAMPOA,3 ROOM,462,CRAWFORD LANE,04 TO 06,60.0,3,1981,63 years 03 months,341000.0,2017,759,4,6,0.6,0.1


## 3. Get walking distance to nearest park